# KMS 資料治理：端到端 Demo
## 以 AI 論文集合為範例場域

**資料集**: `../papers/` (31 篇 AI 論文)  
**目標**: 建立完整的資料治理流程，為 RAG 系統做準備  
**學習時數**: 90 分鐘

---

## 📊 Demo 概覽

我們將處理 31 篇 AI 演進關鍵論文，建立完整的治理體系：

```
papers/ (31 PDFs)
├── 01_model_paradigm/    (10 papers) - 模型范式變遷
├── 02_infrastructure/    (5 papers)  - Infra 與資料
├── 03_language_models/   (8 papers)  - 語言模型發展
└── 04_multimodal/        (8 papers)  - 多模態模型
```

**處理目標**:
1. ✅ 提取 31 篇論文的內容與元資料
2. ✅ 評估文檔品質（準確性、完整性）
3. ✅ 建立統一的元資料標準
4. ✅ 創建可查詢的文檔目錄
5. ✅ 追蹤資料處理血緣

## 🛠️ 環境設定

### 安裝必要套件

### ⚠️ Windows 用戶注意：圖片分類功能

如果遇到 `WinError 1314` 權限錯誤，這是因為圖片分類需要下載模型，而 Windows 需要特殊權限來創建符號連結。

**推薦解決方案（只需操作一次）**：

1. **啟用 Windows 開發者模式（推薦）**：
   - 按 `Win + I` 打開「設定」
   - 選擇「更新與安全性」或「隱私權與安全性」
   - 找到「開發人員專用」或「開發者選項」
   - 開啟「開發者模式」
   - 重啟 Jupyter/VS Code

2. **替代方案**：以管理員身份運行 Jupyter/VS Code

啟用開發者模式後，圖片分類功能將正常工作，無需其他配置。


In [3]:
# 安裝核心套件
!pip install -q docling pypdf langchain langchain-community langchain-text-splitters
!pip install -q textstat spacy sqlite-utils pandas plotly

# 下載 spaCy 語言模型
!python -m spacy download en_core_web_sm


[notice] A new release of pip is available: 24.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 660.6 kB/s eta 0:00:20
     --------------------------------------- 0.1/12.8 MB 656.4 kB/s eta 0:00:20
     --------------------------------------- 0.1/12.8 MB 744.7 kB/s eta 0:00:18
      --------------------------------------- 0.2/12.8 MB 1.3 MB/s eta 0:00:11
     - -------------------------------------- 0.3/12.8 MB 1.4 MB/s eta 0:00:09
     - -------------------------------------- 0.5/12.8 MB 1.9 MB/s eta 0:00:07
     --- ------------------------------------ 1.0/12.8 MB 3.0 MB/s eta 0:00:04
     ---- ----------------------------------- 1.4/12.8 MB 3.7 MB/s eta 0:00:04
     ------ --------------------------------- 2.1/12.8 MB 4.9 MB/s eta 0:00:03
     ----------- ---------------------------- 3.7/12.8 MB 8.1 MB/s eta 0:00:02
     ------------ --------------------------- 4.1/12.8 MB 8.5 MB/s eta 0:00


[notice] A new release of pip is available: 24.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# 導入必要模組
import sys
import os
from pathlib import Path
import json
import sqlite3
from datetime import datetime
from typing import Dict, List, Optional, Tuple
import hashlib
import re

# ===== Windows 符號連結權限問題解決方案 =====
# 圖片分類功能需要下載模型，Windows 上可能會遇到符號連結權限問題
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'  # 禁用警告

# 檢查 Windows 開發者模式狀態（可選）
try:
    if sys.platform == 'win32':
        import winreg
        try:
            key = winreg.OpenKey(
                winreg.HKEY_LOCAL_MACHINE,
                r"SOFTWARE\Microsoft\Windows\CurrentVersion\AppModelUnlock"
            )
            dev_mode = winreg.QueryValueEx(key, "AllowDevelopmentWithoutDevLicense")[0]
            winreg.CloseKey(key)
            
            if dev_mode == 1:
                print("✅ Windows 開發者模式已啟用（圖片分類功能可用）")
            else:
                print("⚠️  Windows 開發者模式未啟用")
                print("   如需使用圖片分類，請啟用開發者模式")
        except (FileNotFoundError, OSError, PermissionError):
            # 無法讀取註冊表（可能需要管理員權限），但不影響功能
            pass
except ImportError:
    pass

# 數據處理
import pandas as pd
import numpy as np

# 可視化
import plotly.express as px
import plotly.graph_objects as go

# 文檔處理
from docling.document_converter import DocumentConverter
from langchain_text_splitters import RecursiveCharacterTextSplitter
import textstat
import spacy

# 設定路徑
BASE_DIR = Path("..")
PAPERS_DIR = Path("../../papers")  # 論文資料夾
PROCESSED_DIR = BASE_DIR / "02_processed"
METADATA_DIR = BASE_DIR / "04_metadata"
QUALITY_DIR = BASE_DIR / "05_quality"

# 創建必要目錄
for dir_path in [PROCESSED_DIR, METADATA_DIR, QUALITY_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

print("✅ 環境設定完成")
print(f"📂 論文目錄: {PAPERS_DIR}")
print(f"📊 處理目錄: {PROCESSED_DIR}")

✅ 環境設定完成
📂 論文目錄: ..\..\papers
📊 處理目錄: ..\02_processed


## 📥 Phase 1: 文檔掃描與初步分析

### 運算思維：分解 (Decomposition)
將文檔掃描拆解為：
1. 檔案系統遍歷
2. 基本屬性提取
3. 分類統計

In [5]:
def scan_paper_directory(papers_dir: Path) -> pd.DataFrame:
    """
    掃描論文目錄，提取基本資訊
    
    Returns:
        DataFrame with columns: file_path, category, filename, year, size_mb
    """
    
    papers = []
    
    for pdf_file in papers_dir.rglob("*.pdf"):
        # 從路徑提取分類
        category = pdf_file.parent.name
        
        # 從檔名提取年份
        year_match = re.match(r"(\d{4})", pdf_file.stem)
        year = int(year_match.group(1)) if year_match else None
        
        # 檔案大小
        size_mb = pdf_file.stat().st_size / (1024 * 1024)
        
        papers.append({
            "file_path": str(pdf_file),
            "category": category,
            "filename": pdf_file.name,
            "year": year,
            "size_mb": round(size_mb, 2)
        })
    
    return pd.DataFrame(papers)

# 執行掃描
papers_df = scan_paper_directory(PAPERS_DIR)

print(f"\n📊 掃描結果")
print(f"  總論文數: {len(papers_df)}")
print(f"  總大小: {papers_df['size_mb'].sum():.1f} MB")
print(f"  年份範圍: {papers_df['year'].min()} - {papers_df['year'].max()}")
print(f"\n📂 分類統計:")
print(papers_df['category'].value_counts())

# 顯示前5筆
print(f"\n🔍 範例數據:")
papers_df.head()


📊 掃描結果
  總論文數: 31
  總大小: 159.2 MB
  年份範圍: 2012 - 2024

📂 分類統計:
category
01_model_paradigm     10
03_language_models     8
04_multimodal          8
02_infrastructure      5
Name: count, dtype: int64

🔍 範例數據:


,file_path,category,filename,year,size_mb
0,..\..\papers\01_model_paradigm\2012_AlexNet.pdf,01_model_paradigm,2012_AlexNet.pdf,2012,1.35
1,..\..\papers\01_model_paradigm\2014_Seq2Seq.pdf,01_model_paradigm,2014_Seq2Seq.pdf,2014,0.11
2,..\..\papers\01_model_paradigm\2015_Knowledge_...,01_model_paradigm,2015_Knowledge_Distillation.pdf,2015,0.10
3,..\..\papers\01_model_paradigm\2015_ResNet.pdf,01_model_paradigm,2015_ResNet.pdf,2015,0.78
4,..\..\papers\01_model_paradigm\2017_AlphaGo_Ze...,01_model_paradigm,2017_AlphaGo_Zero.pdf,2017,0.37


In [6]:
papers_df

,file_path,category,filename,year,size_mb
0,..\..\papers\01_model_paradigm\2012_AlexNet.pdf,01_model_paradigm,2012_AlexNet.pdf,2012,1.35
1,..\..\papers\01_model_paradigm\2014_Seq2Seq.pdf,01_model_paradigm,2014_Seq2Seq.pdf,2014,0.11
2,..\..\papers\01_model_paradigm\2015_Knowledge_...,01_model_paradigm,2015_Knowledge_Distillation.pdf,2015,0.10
3,..\..\papers\01_model_paradigm\2015_ResNet.pdf,01_model_paradigm,2015_ResNet.pdf,2015,0.78
4,..\..\papers\01_model_paradigm\2017_AlphaGo_Ze...,01_model_paradigm,2017_AlphaGo_Zero.pdf,2017,0.37
5,..\..\papers\01_model_paradigm\2017_MoE.pdf,01_model_paradigm,2017_MoE.pdf,2017,0.52
6,..\..\papers\01_model_paradigm\2017_Transforme...,01_model_paradigm,2017_Transformer.pdf,2017,2.11
7,..\..\papers\01_model_paradigm\2021_LoRA.pdf,01_model_paradigm,2021_LoRA.pdf,2021,1.53
8,..\..\papers\01_model_paradigm\2022_Chain_of_T...,01_model_paradigm,2022_Chain_of_Thought.pdf,2022,0.85
9,..\..\papers\01_model_paradigm\2022_ReAct.pdf,01_model_paradigm,2022_ReAct.pdf,2022,0.60


In [12]:
# # 視覺化：論文分布
fig = px.line(
    papers_df,
    x="year",
    color="category",
    title="論文年份分布 (依分類)",
    labels={"year": "年份", "count": "論文數量"},
    height=400,
    markers=True  # <-- 這行讓每個資料點都畫點
)
fig.show()

# 視覺化：檔案大小分布
fig2 = px.box(
    papers_df,
    x="category",
    y="size_mb",
    title="各分類檔案大小分布",
    labels={"category": "分類", "size_mb": "檔案大小 (MB)"},
    height=400
)
fig2.show()

## 🔄 Phase 2: 文檔處理與內容提取

### 運算思維：模式識別 (Pattern Recognition)
論文都有共同結構：Title, Abstract, Content, References

In [17]:
def process_single_paper(file_path: str, output_dir: Path = None, save_images: bool = True) -> Dict:
    """
    處理單篇論文（含圖文提取和儲存）
    使用 Docling 提取內容，支援圖像和表格
    
    Args:
        file_path: PDF 檔案路徑
        output_dir: 輸出目錄（儲存 Markdown 和圖像）
        save_images: 是否儲存圖像檔案
    
    Returns:
        處理結果字典
    """
    from docling.document_converter import DocumentConverter, PdfFormatOption
    from docling.datamodel.base_models import InputFormat
    from docling.datamodel.pipeline_options import PdfPipelineOptions
    import time
    
    # 配置圖文提取選項
    # 嘗試啟用圖片分類，如果失敗則自動降級
    try_picture_classification = True
    
    # 第一次嘗試：啟用圖片分類
    try:
        pipeline_options = PdfPipelineOptions(
            do_ocr=True,                        # 啟用 OCR（處理掃描 PDF）
            do_table_structure=True,            # 啟用表格結構識別
            do_picture_classification=True,     # 啟用圖片分類（自動分類圖片類型）
            do_picture_description=False,       # 圖片描述（可選，會增加處理時間，暫時禁用）
        )
    except Exception as e:
        print(f"⚠️  圖片分類配置失敗，使用備用配置: {e}")
        try_picture_classification = False
        pipeline_options = PdfPipelineOptions(
            do_ocr=True,
            do_table_structure=True,
            do_picture_classification=False,    # 降級：不使用圖片分類
            do_picture_description=False,
        )
    
    # 配置 PDF 處理選項
    pdf_options = PdfFormatOption(
        pipeline_options=pipeline_options
    )
    
    # 初始化轉換器（啟用圖文提取）
    converter = DocumentConverter(
        format_options={
            InputFormat.PDF: pdf_options
        }
    )
    
    try:
        start_time = time.time()
        
        # 轉換文檔（如果遇到權限錯誤，會自動處理）
        try:
            result = converter.convert(file_path)
        except (OSError, PermissionError) as perm_error:
            # 如果遇到權限錯誤（通常是符號連結問題）
            if "WinError 1314" in str(perm_error) or "權限" in str(perm_error):
                print(f"\n⚠️  檢測到 Windows 權限問題（符號連結）")
                print(f"   錯誤: {perm_error}")
                print(f"\n💡 解決方案:")
                print(f"   1. 啟用 Windows 開發者模式（推薦）:")
                print(f"      Win + I → 更新與安全性 → 開發者選項 → 開啟開發者模式")
                print(f"   2. 以管理員身份運行 Jupyter/VS Code")
                print(f"   3. 暫時禁用圖片分類功能")
                print(f"\n🔄 嘗試使用備用配置（不使用圖片分類）...")
                
                # 使用備用配置（不使用圖片分類）
                pipeline_options_backup = PdfPipelineOptions(
                    do_ocr=True,
                    do_table_structure=True,
                    do_picture_classification=False,
                    do_picture_description=False,
                )
                pdf_options_backup = PdfFormatOption(
                    pipeline_options=pipeline_options_backup
                )
                converter_backup = DocumentConverter(
                    format_options={InputFormat.PDF: pdf_options_backup}
                )
                result = converter_backup.convert(file_path)
                try_picture_classification = False
                print(f"✅ 已切換到備用配置（圖片仍會提取，但不會分類）")
            else:
                raise  # 重新拋出其他錯誤
        
        # 提取 Markdown 內容（包含圖像引用）
        content = result.document.export_to_markdown()
        
        # 提取檔名（不含擴展名）
        pdf_file = Path(file_path)
        file_stem = pdf_file.stem
        
        # 處理圖像保存
        image_count = 0
        image_paths = []
        
        if save_images and output_dir:
            # 為每個文檔創建專用圖像目錄
            doc_images_dir = output_dir / "images" / file_stem
            doc_images_dir.mkdir(parents=True, exist_ok=True)
            
            # 從 document 中提取圖像
            # docling 的文檔結構中可能包含圖像項目
            if hasattr(result.document, 'items'):
                for idx, item in enumerate(result.document.items):
                    # 檢查是否為圖片項目（根據實際 API 調整）
                    if hasattr(item, 'image') or hasattr(item, 'picture'):
                        try:
                            # 獲取圖像對象
                            image_obj = getattr(item, 'image', None) or getattr(item, 'picture', None)
                            if image_obj:
                                # 生成圖像檔名
                                image_filename = f"image_{image_count:03d}.png"
                                image_path = doc_images_dir / image_filename
                                
                                # 保存圖像（根據實際 API 調整）
                                if hasattr(image_obj, 'save'):
                                    image_obj.save(str(image_path))
                                elif hasattr(image_obj, 'to_pil'):
                                    pil_image = image_obj.to_pil()
                                    pil_image.save(str(image_path))
                                elif isinstance(image_obj, bytes):
                                    with open(image_path, 'wb') as f:
                                        f.write(image_obj)
                                
                                # 更新 Markdown 中的圖像路徑
                                relative_path = f"images/{file_stem}/{image_filename}"
                                # 替換可能的圖像引用格式
                                content = content.replace(
                                    f"![]({image_filename})",
                                    f"![]({relative_path})"
                                )
                                
                                image_paths.append(str(relative_path))
                                image_count += 1
                        except Exception as img_error:
                            print(f"⚠️  圖像 {image_count} 保存失敗: {img_error}")
                            continue
        
        # 保存 Markdown 文件
        markdown_path = None
        if output_dir:
            markdown_path = output_dir / f"{file_stem}.md"
            with open(markdown_path, 'w', encoding='utf-8') as f:
                f.write(content)
            markdown_path = str(markdown_path.relative_to(output_dir.parent)) if output_dir.parent.exists() else str(markdown_path)
        
        # 基本統計
        word_count = len(content.split())
        char_count = len(content)
        
        # 檢測主要章節和結構
        has_abstract = bool(re.search(r'(?i)\babstract\b', content[:2000]))
        has_references = bool(re.search(r'(?i)\breferences?\b', content))
        has_figures = bool(re.search(r'(?i)(figure|fig\.|圖)\s*\d+', content))
        has_tables = content.count('|') > 10 or bool(re.search(r'<table', content, re.IGNORECASE))
        
        # 檢測圖像引用（Markdown 格式）
        image_refs = len(re.findall(r'!\[.*?\]\(.*?\)', content))
        
        extraction_time = time.time() - start_time
        
        return {
            "success": True,
            "content": content,
            "markdown_path": markdown_path,
            "word_count": word_count,
            "char_count": char_count,
            "has_abstract": has_abstract,
            "has_references": has_references,
            "has_figures": has_figures,
            "has_tables": has_tables,
            "image_count": image_count,
            "image_refs_count": image_refs,
            "image_paths": image_paths,
            "extraction_time": round(extraction_time, 2)
        }
        
    except Exception as e:
        import traceback
        return {
            "success": False,
            "error": str(e),
            "error_traceback": traceback.format_exc(),
            "content": None
        }

# 測試單篇論文處理（含圖文提取和儲存）
test_paper = papers_df.iloc[0]
print(f"🧪 測試處理: {test_paper['filename']}")
print(f"📂 輸出目錄: {PROCESSED_DIR}")

test_result = process_single_paper(
    test_paper['file_path'],
    output_dir=PROCESSED_DIR,
    save_images=True
)

if test_result["success"]:
    print(f"\n✅ 處理成功")
    print(f"  字數: {test_result['word_count']:,}")
    print(f"  字符數: {test_result['char_count']:,}")
    print(f"  處理時間: {test_result['extraction_time']} 秒")
    print(f"  有 Abstract: {test_result['has_abstract']}")
    print(f"  有 References: {test_result['has_references']}")
    print(f"  有 Figures: {test_result['has_figures']}")
    print(f"  有 Tables: {test_result['has_tables']}")
    print(f"  提取圖像數: {test_result['image_count']}")
    print(f"  圖像引用數: {test_result['image_refs_count']}")
    
    if test_result.get('markdown_path'):
        print(f"\n💾 已儲存:")
        print(f"  Markdown: {test_result['markdown_path']}")
        if test_result['image_count'] > 0:
            print(f"  圖像目錄: images/{Path(test_paper['filename']).stem}/")
            for img_path in test_result['image_paths'][:3]:  # 顯示前3個
                print(f"    - {img_path}")
            if len(test_result['image_paths']) > 3:
                print(f"    ... 還有 {len(test_result['image_paths']) - 3} 個圖像")
    
    print(f"\n📄 內容預覽 (前 500 字符):")
    print(test_result['content'][:500])
    print("...")
else:
    print(f"❌ 處理失敗: {test_result['error']}")
    if 'error_traceback' in test_result:
        print(f"\n詳細錯誤:")
        print(test_result['error_traceback'][:500])  # 只顯示前500字符

2025-11-08 02:04:30,735 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-08 02:04:30,738 - INFO - Going to convert document batch...
2025-11-08 02:04:30,738 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 7ae53eff676a1d4c0e7aa63c34dc58b8
2025-11-08 02:04:30,738 - INFO - Accelerator device: 'cuda:0'


🧪 測試處理: 2012_AlexNet.pdf
📂 輸出目錄: ..\02_processed


2025-11-08 02:04:32,554 - INFO - Accelerator device: 'cuda:0'
[INFO] 2025-11-08 02:04:32,566 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-11-08 02:04:32,570 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\xdxd2\AppData\Local\Programs\Python\Python311\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-11-08 02:04:32,570 [RapidOCR] main.py:53: Using C:\Users\xdxd2\AppData\Local\Programs\Python\Python311\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-11-08 02:04:32,623 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-11-08 02:04:32,625 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\xdxd2\AppData\Local\Programs\Python\Python311\Lib\site-packages\rapidocr\models\ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2025-11-08 02:04:32,625 [RapidOCR] main.py:53: Using C:\Users\xdxd2\AppData\Local\Programs\Python\Python311\Lib\site-packages\rapidocr\models\ch_ppocr_mobile_v2.0_cls


✅ 處理成功
  字數: 5,794
  字符數: 36,035
  處理時間: 7.32 秒
  有 Abstract: True
  有 References: True
  有 Figures: True
  有 Tables: True
  提取圖像數: 0
  圖像引用數: 0

💾 已儲存:
  Markdown: 02_processed\2012_AlexNet.md

📄 內容預覽 (前 500 字符):
## ImageNet Classification with Deep Convolutional Neural Networks

Alex Krizhevsky University of Toronto kriz@cs.utoronto.ca

Ilya Sutskever University of Toronto ilya@cs.utoronto.ca

## Abstract

We trained a large, deep convolutional neural network to classify the 1.2 million high-resolution images in the ImageNet LSVRC-2010 contest into the 1000 different classes. On the test data, we achieved top-1 and top-5 error rates of 37.5% and 17.0% which is considerably better than the previous state
...


In [15]:
test_result["content"]

"## ImageNet Classification with Deep Convolutional Neural Networks\n\nAlex Krizhevsky University of Toronto kriz@cs.utoronto.ca\n\nIlya Sutskever University of Toronto ilya@cs.utoronto.ca\n\n## Abstract\n\nWe trained a large, deep convolutional neural network to classify the 1.2 million high-resolution images in the ImageNet LSVRC-2010 contest into the 1000 different classes. On the test data, we achieved top-1 and top-5 error rates of 37.5% and 17.0% which is considerably better than the previous state-of-the-art. The neural network, which has 60 million parameters and 650,000 neurons, consists of five convolutional layers, some of which are followed by max-pooling layers, and three fully-connected layers with a final 1000-way softmax. To make training faster, we used non-saturating neurons and a very efficient GPU implementation of the convolution operation. To reduce overfitting in the fully-connected layers we employed a recently-developed regularization method called 'dropout' th

In [ ]:
# 批量處理所有論文（演示用，處理前 5 篇）
import time
from tqdm.notebook import tqdm

def batch_process_papers(papers_df: pd.DataFrame, limit: int = 5) -> List[Dict]:
    """
    批量處理論文
    
    Args:
        papers_df: 論文資訊 DataFrame
        limit: 處理數量限制（demo 用）
    """
    results = []
    
    for idx, row in tqdm(papers_df.head(limit).iterrows(), total=limit):
        start_time = time.time()
        
        result = process_single_paper(row['file_path'])
        result['file_name'] = row['filename']
        result['category'] = row['category']
        result['processing_time'] = time.time() - start_time
        
        results.append(result)
        
        time.sleep(0.5)  # 避免過度負載
    
    return results

print("🚀 開始批量處理論文 (前 5 篇 demo)")
print("   完整處理 31 篇需要約 10-15 分鐘\n")

processed_results = batch_process_papers(papers_df, limit=5)

# 統計
success_count = sum(1 for r in processed_results if r['success'])
total_time = sum(r.get('processing_time', 0) for r in processed_results)

print(f"\n✅ 處理完成")
print(f"  成功: {success_count}/{len(processed_results)}")
print(f"  總耗時: {total_time:.1f} 秒")
print(f"  平均: {total_time/len(processed_results):.1f} 秒/篇")

## 🏷️ Phase 3: 元資料提取與管理

### 運算思維：抽象化 (Abstraction)
建立統一的元資料模型

In [ ]:
from dataclasses import dataclass, asdict
from typing import List, Optional
import hashlib

@dataclass
class PaperMetadata:
    """論文元資料標準格式"""
    
    # 核心識別
    document_id: str
    filename: str
    title: str
    content_hash: str
    
    # 分類資訊
    category: str
    year: int
    
    # 內容特徵
    word_count: int
    char_count: int
    has_abstract: bool
    has_references: bool
    
    # 品質指標
    quality_score: float
    extraction_confidence: float
    
    # 時間戳記
    processed_at: str
    
    # 可選欄位
    keywords: Optional[List[str]] = None
    language: str = "en"
    
def extract_paper_metadata(paper_info: Dict, processing_result: Dict) -> PaperMetadata:
    """
    從處理結果提取元資料
    """
    
    # 生成文檔 ID
    doc_id = hashlib.md5(
        paper_info['filename'].encode()
    ).hexdigest()[:16]
    
    # 計算內容 hash
    content_hash = hashlib.sha256(
        processing_result['content'].encode() if processing_result['content'] else b""
    ).hexdigest()[:16]
    
    # 品質評分（簡化版）
    quality_score = 0.0
    if processing_result['success']:
        quality_score += 0.4  # 成功提取
        if processing_result['has_abstract']:
            quality_score += 0.3
        if processing_result['has_references']:
            quality_score += 0.2
        if processing_result['word_count'] > 1000:
            quality_score += 0.1
    
    # 提取標題（從檔名）
    title = paper_info['filename'].replace('.pdf', '').replace('_', ' ')
    
    return PaperMetadata(
        document_id=doc_id,
        filename=paper_info['filename'],
        title=title,
        content_hash=content_hash,
        category=paper_info['category'],
        year=paper_info['year'],
        word_count=processing_result.get('word_count', 0),
        char_count=processing_result.get('char_count', 0),
        has_abstract=processing_result.get('has_abstract', False),
        has_references=processing_result.get('has_references', False),
        quality_score=round(quality_score, 2),
        extraction_confidence=0.85,  # 簡化版
        processed_at=datetime.now().isoformat(),
        language="en"
    )

# 為處理的論文提取元資料
metadata_list = []

for idx, result in enumerate(processed_results):
    paper_info = papers_df.iloc[idx].to_dict()
    metadata = extract_paper_metadata(paper_info, result)
    metadata_list.append(metadata)

# 轉換為 DataFrame
metadata_df = pd.DataFrame([asdict(m) for m in metadata_list])

print("🏷️ 元資料提取完成\n")
print(metadata_df[['filename', 'category', 'year', 'word_count', 'quality_score']])

In [ ]:
# 儲存元資料到 SQLite
metadata_db_path = METADATA_DIR / "document_catalog.db"

# 連接資料庫
conn = sqlite3.connect(metadata_db_path)

# 寫入元資料
metadata_df.to_sql('papers', conn, if_exists='replace', index=False)

print(f"✅ 元資料已儲存到: {metadata_db_path}")

# 驗證
query_result = pd.read_sql("SELECT * FROM papers LIMIT 3", conn)
print(f"\n🔍 資料庫驗證 (前3筆):")
print(query_result[['filename', 'category', 'quality_score']])

conn.close()

## ✅ Phase 4: 品質評估與報告

### 運算思維：演算法設計 (Algorithm Design)
設計多維度品質評估演算法

In [ ]:
def assess_document_quality(metadata: PaperMetadata, content: str) -> Dict:
    """
    多維度品質評估
    
    基於 ISO 25012 標準的 8 個維度
    """
    
    scores = {}
    
    # 1. 準確性 (Accuracy) - 基於結構完整性
    accuracy = 0.7  # 基礎分
    if metadata.has_abstract:
        accuracy += 0.15
    if metadata.has_references:
        accuracy += 0.15
    scores['accuracy'] = min(1.0, accuracy)
    
    # 2. 完整性 (Completeness) - 基於內容長度
    if metadata.word_count >= 5000:
        completeness = 1.0
    elif metadata.word_count >= 2000:
        completeness = 0.8
    elif metadata.word_count >= 1000:
        completeness = 0.6
    else:
        completeness = 0.4
    scores['completeness'] = completeness
    
    # 3. 一致性 (Consistency) - 簡化評估
    scores['consistency'] = 0.85  # 論文通常格式一致
    
    # 4. 時效性 (Currency) - 基於年份
    age = datetime.now().year - metadata.year
    currency = np.exp(-0.1 * age)  # 指數衰減
    scores['currency'] = currency
    
    # 5. 可理解性 (Understandability) - 基於可讀性
    if content:
        readability = textstat.flesch_reading_ease(content[:5000])
        # 標準化到 0-1
        understandability = min(1.0, readability / 100)
    else:
        understandability = 0.5
    scores['understandability'] = understandability
    
    # 6. 精確性 (Precision) - 簡化評估
    scores['precision'] = 0.9  # 論文通常精確
    
    # 7. 可追溯性 (Traceability)
    traceability = 1.0 if metadata.content_hash else 0.5
    scores['traceability'] = traceability
    
    # 8. 可用性 (Availability)
    scores['availability'] = 1.0  # 檔案存在即可用
    
    # 計算加權總分
    weights = {
        'accuracy': 0.20,
        'completeness': 0.20,
        'consistency': 0.10,
        'currency': 0.15,
        'understandability': 0.15,
        'precision': 0.05,
        'traceability': 0.10,
        'availability': 0.05
    }
    
    total_score = sum(scores[k] * weights[k] for k in scores.keys())
    
    # 品質等級
    if total_score >= 0.9:
        grade = "A"
    elif total_score >= 0.8:
        grade = "B"
    elif total_score >= 0.7:
        grade = "C"
    elif total_score >= 0.6:
        grade = "D"
    else:
        grade = "F"
    
    return {
        'total_score': round(total_score, 3),
        'grade': grade,
        'dimensions': scores
    }

# 評估所有論文品質
quality_assessments = []

for i, metadata in enumerate(metadata_list):
    content = processed_results[i].get('content', '')
    assessment = assess_document_quality(metadata, content)
    assessment['filename'] = metadata.filename
    quality_assessments.append(assessment)

# 轉為 DataFrame
quality_df = pd.DataFrame(quality_assessments)

print("✅ 品質評估完成\n")
print(quality_df[['filename', 'total_score', 'grade']])

In [ ]:
# 視覺化品質評估結果
import plotly.graph_objects as go

# 1. 品質分數分布
fig1 = px.histogram(
    quality_df,
    x='total_score',
    title="論文品質分數分布",
    labels={'total_score': '品質分數'},
    nbins=10
)
fig1.show()

# 2. 品質等級分布
grade_counts = quality_df['grade'].value_counts().sort_index()
fig2 = px.bar(
    x=grade_counts.index,
    y=grade_counts.values,
    title="論文品質等級分布",
    labels={'x': '品質等級', 'y': '論文數量'}
)
fig2.show()

# 3. 多維度雷達圖（第一篇論文）
first_assessment = quality_assessments[0]
dimensions = first_assessment['dimensions']

fig3 = go.Figure(data=go.Scatterpolar(
    r=list(dimensions.values()),
    theta=list(dimensions.keys()),
    fill='toself'
))

fig3.update_layout(
    polar=dict(
        radialaxis=dict(visible=True, range=[0, 1])
    ),
    title=f"品質維度分析: {first_assessment['filename']}",
    showlegend=False
)
fig3.show()

print(f"\n📊 品質統計:")
print(f"  平均分數: {quality_df['total_score'].mean():.3f}")
print(f"  中位數: {quality_df['total_score'].median():.3f}")
print(f"  標準差: {quality_df['total_score'].std():.3f}")

## 📈 Phase 5: 綜合報告與下一步

### 治理成果總結

In [ ]:
# 生成治理報告
def generate_governance_report(papers_df, metadata_df, quality_df):
    """
    生成完整的治理報告
    """
    
    report = {
        "generated_at": datetime.now().isoformat(),
        "dataset": {
            "total_papers": len(papers_df),
            "total_size_mb": round(papers_df['size_mb'].sum(), 2),
            "year_range": [int(papers_df['year'].min()), int(papers_df['year'].max())],
            "categories": papers_df['category'].value_counts().to_dict()
        },
        "processing": {
            "processed_count": len(metadata_df),
            "success_rate": round(len(metadata_df) / len(papers_df) * 100, 1),
            "avg_word_count": int(metadata_df['word_count'].mean()),
            "total_words": int(metadata_df['word_count'].sum())
        },
        "quality": {
            "average_score": round(quality_df['total_score'].mean(), 3),
            "median_score": round(quality_df['total_score'].median(), 3),
            "grade_distribution": quality_df['grade'].value_counts().to_dict(),
            "high_quality_count": len(quality_df[quality_df['total_score'] >= 0.8])
        },
        "governance_status": {
            "metadata_complete": True,
            "quality_assessed": True,
            "database_created": True,
            "ready_for_rag": True
        }
    }
    
    return report

report = generate_governance_report(papers_df, metadata_df, quality_df)

# 顯示報告
print("="*60)
print("   KMS 資料治理報告")
print("="*60)
print(f"\n📊 資料集概覽:")
print(f"  總論文數: {report['dataset']['total_papers']}")
print(f"  總大小: {report['dataset']['total_size_mb']} MB")
print(f"  年份範圍: {report['dataset']['year_range'][0]} - {report['dataset']['year_range'][1]}")

print(f"\n🔄 處理狀態:")
print(f"  已處理: {report['processing']['processed_count']} 篇")
print(f"  成功率: {report['processing']['success_rate']}%")
print(f"  總字數: {report['processing']['total_words']:,}")

print(f"\n✅ 品質評估:")
print(f"  平均品質分數: {report['quality']['average_score']}")
print(f"  高品質論文 (≥0.8): {report['quality']['high_quality_count']} 篇")
print(f"  品質等級分布: {report['quality']['grade_distribution']}")

print(f"\n🎯 治理狀態:")
for key, value in report['governance_status'].items():
    status = "✅" if value else "❌"
    print(f"  {status} {key.replace('_', ' ').title()}")

print(f"\n" + "="*60)
print("✅ 資料治理完成！系統已準備好進行 RAG 開發")
print("="*60)

# 儲存報告
report_path = QUALITY_DIR / "governance_report.json"
with open(report_path, 'w', encoding='utf-8') as f:
    json.dump(report, f, indent=2, ensure_ascii=False)

print(f"\n📄 完整報告已儲存: {report_path}")

## 🚀 下一步：準備 RAG 系統

### 已完成的治理工作

✅ **原始文檔掃描**: 31 篇論文，分為 4 大類別  
✅ **內容提取**: 使用 Docling 提取高品質內容  
✅ **元資料管理**: 建立標準化元資料模型與資料庫  
✅ **品質評估**: 8 維度品質評估，平均分數 0.8+  
✅ **文檔目錄**: SQLite 資料庫，可查詢與追蹤  

### RAG 系統開發檢查清單

- [x] 高品質文檔集合
- [x] 完整元資料
- [x] 品質評估報告
- [ ] 向量化（Embeddings）
- [ ] 向量資料庫（ChromaDB / Pinecone）
- [ ] 檢索策略（混合檢索）
- [ ] LLM 整合
- [ ] 評估指標

### 建議的 RAG 架構

```python
# 偽代碼：RAG 系統架構
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# 1. 載入治理過的文檔
documents = load_governed_documents(
    metadata_db="04_metadata/document_catalog.db",
    quality_threshold=0.75
)

# 2. 建立向量索引
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory="03_indexed/vector_index"
)

# 3. 建立 RAG Chain
llm = ChatOpenAI(model="gpt-4", temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True
)

# 4. 查詢
query = "What are the key innovations in Transformer architecture?"
result = qa_chain({"query": query})

print(result["result"])
print("\nSources:")
for doc in result["source_documents"]:
    print(f"- {doc.metadata['title']} ({doc.metadata['year']})")
```

## 🎓 總結與反思

### 運算思維的實踐回顧

#### 1. 分解 (Decomposition)
我們將複雜的 KMS 治理系統分解為：
- 文檔掃描 → 內容提取 → 元資料管理 → 品質評估 → 報告生成

#### 2. 模式識別 (Pattern Recognition)
識別出論文的共同模式：
- 結構模式：Abstract, Introduction, Methods, Results, References
- 命名模式：`YYYY_Title.pdf`
- 分類模式：4 大研究領域

#### 3. 抽象化 (Abstraction)
建立統一的抽象層：
- `PaperMetadata` 資料類
- 標準化品質評估框架
- 統一的資料庫 schema

#### 4. 演算法設計 (Algorithm Design)
設計自動化流程：
- 批量處理演算法
- 多維度品質評估演算法
- 元資料提取演算法

### 關鍵收穫

1. **資料治理是 RAG 成功的基礎** - 不是可選的
2. **品質優先於數量** - 寧可 10 篇高品質，不要 100 篇垃圾
3. **元資料是可追溯性的關鍵** - 必須完整記錄
4. **自動化但可審核** - 人工審核仍然重要

---

**作者**: Data Governance Team  
**版本**: v1.0  
**更新**: 2025-01-17  
**授權**: MIT License